# Calculations to design the questionnaire

In [12]:
import numpy as np
from scipy import stats
from scipy import special
import matplotlib.pyplot as plt
import math

### Geometry

In [11]:
total_height = 9
crest_legth = 4
slope = 3
angle_slope = math.atan(1/3)
# print(angle_slope*180/np.pi)
slope_length = 9/math.sin(angle_slope)
# print(slope_length)

28.460498941515414


### Boundary conditions: overtopping events at the beginning of the slope

It should be noted that the equations below should be applied considering the breaking/non breaking nature of the waves, their experimental ranges and values of influence parameters. Here, they are used as a fast tool to assess the pre-design of a 2D 'experiment' so influence parameters are set to 1 and non-breaking are assumed.

In [39]:
def steepness(Hs, Tm0):
    return Hs/((9.81*Tm0**2)/2/np.pi)

def iribarren(Hs, Tm0, angle_slope):
    return np.tan(angle_slope)/np.sqrt(steepness(Hs, Tm0))

def params_distr_vols(q, Hs, Tm0, Pov):
    Tm = 1.1*Tm0
    b = 0.73 + 55*(q/(9.81*Hs*Tm0))**(0.8)
    a = (1/special.gamma(1 + 1/b))*(q*Tm/Pov)
    return a, b

def pov(Rc, Ru):
    return np.exp(-((np.sqrt(-np.log(0.02)))*Rc/Ru)**2)

def Ru(Hs, Tm0, angle_slope):
    ru = 1.65*Hs*1*1*1*iribarren(Hs, Tm0, angle_slope)
    max_value = Hs*1*1*(4-1.5/iribarren(Hs, Tm0, angle_slope))
    return min(ru, max_value)

def mean_discharge(Hs, Tm0, Rc, angle_slope):
    estimation = 0.023*np.sqrt(9.81*Hs**3)*1*iribarren(Hs, Tm0, angle_slope)*np.exp(-(2.7*Rc/(Hs*iribarren(Hs, Tm0, angle_slope)))**1.3)
    max_value = 0.09*np.sqrt(9.81*Hs**3)*np.exp(-(1.5*Rc/Hs)**1.3)
    return min(estimation, max_value)

def weibull_cdf(V, a, b):
    return 1 - np.exp(-(V/a)**b)

def weibull_icdf(P, a, b):
    return a*(-np.log(1-P))**(1/b)

def individual_events(Hs, Tm0, Rc, angle_slope, Pov, n_samples):
    q_mean = mean_discharge(Hs, Tm0, Rc, angle_slope)
    a, b = params_distr_vols(q_mean, Hs, Tm0, Pov)
    unif_samples = stats.uniform.rvs(0, 1, size = n_samples)
    V_samples = weibull_icdf(unif_samples, a, b)
    vel_samples = 4.5*V_samples**0.3
    h_samples = 0.133*V_samples**0.5
    T_samples = 3.9*V_samples**0.46
    samples = np.vstack((V_samples, vel_samples, h_samples, T_samples)).T
    return samples

In [38]:
Hs = 2.5
Tm0 = 7
water_level = 6
Rc = total_height - water_level

# mean_discharge(Hs, Tm0, Rc, angle_slope)/Hs
# Ru(Hs, Tm0, angle_slope)
# pov(Rc, Ru(Hs, Tm0, angle_slope))
# iribarren(Hs, Tm0, angle_slope)

# params_distr_vols(mean_discharge(Hs, Tm0, Rc, angle_slope), Hs, Tm0, pov(Rc, Ru(Hs, Tm0, angle_slope)))

# individual_events(Hs, Tm0, Rc, angle_slope, Pov, n_samples)

0.5441415592273666

In [43]:
# Input variables
Hs = 2.5
Tm0 = 7
water_level = 6
Rc = total_height - water_level
t_storm = 6 #hours of storm

#Number of overtopping events
n_events = np.round((t_storm*3600/(1.1*Tm0))*pov(Rc, Ru(Hs, Tm0, angle_slope)))

#Generate overtopping events
n_samples = int(n_events)
samples = individual_events(Hs, Tm0, Rc, angle_slope, pov(Rc, Ru(Hs, Tm0, angle_slope)), n_samples)


array([[0.23907309, 2.92935309, 0.06503048, 2.01924366],
       [0.23068352, 2.89812746, 0.06387927, 1.98633376],
       [0.53451111, 3.7290614 , 0.09723665, 2.92364546],
       ...,
       [0.07613743, 2.07822223, 0.03669871, 1.19288835],
       [0.85041187, 4.286485  , 0.12264965, 3.61987932],
       [1.08203258, 4.60770446, 0.13834766, 4.04403739]])

### Flow characteristics along the crest and slope: 1D shallow water equations